 Curve fitting - Non-Linear Least square fitting


In [ ]:
import numpy as np

# Extracted experimental data points from the table
v_exp = np.array([1, 10, 100, 1000])
p_exp = np.array([0.1, 0.5, 1.0, 1.5, 2.0])
mu_exp = np.array([
    [0.7296, 0.5748, 0.4931, 0.3307, 0.4274],
    [1.0348, 0.7191, 0.5843, 0.5019, 0.4944],
    [1.0348, 0.9028, 0.7256, 0.5767, 0.5393],
    [1.325, 0.8516, 0.7012, 0.5661, 0.5569]
])

# Flatten the arrays to create input vectors for v_exp and p_exp
v_exp_flat = np.repeat(v_exp, len(p_exp))
p_exp_flat = np.tile(p_exp, len(v_exp))
mu_exp_flat = mu_exp.flatten()

# Initial guesses for parameters
a, b, c = 1.0, 1.0, 1.0
n, m = 1.0, 1.0
alpha, beta = 1.0, 1.0

# Define the Huemer model function
def huemer_model(v, p, a, b, c, n, m, alpha, beta):
    return (alpha * np.abs(p)**(n-1) + beta) / (a + np.abs(v)**(1/m) / b + c / np.abs(v)**(2/m))

# Least squares error function
def least_squares_error(params, mu_exp, v_exp, p_exp):
    a, b, c, n, m, alpha, beta = params
    mu_pred = huemer_model(v_exp, p_exp, a, b, c, n, m, alpha, beta)
    error = np.sum((mu_exp - mu_pred)**2)
    return error

# Iteratively update parameters to minimize the error
def update_parameters(mu_exp, v_exp, p_exp, params, learning_rate=0.001, iterations=10000):
    for _ in range(iterations):
        gradients = np.zeros_like(params)
        for i in range(len(params)):
            params[i] += 1e-5  # Small perturbation
            error1 = least_squares_error(params, mu_exp, v_exp, p_exp)
            params[i] -= 2e-5  # Small perturbation in the opposite direction
            error2 = least_squares_error(params, mu_exp, v_exp, p_exp)
            gradients[i] = (error1 - error2) / (2 * 1e-5)
            params[i] += 1e-5  # Restore the parameter

        params -= learning_rate * gradients
    return params

# Initial parameter array
params = np.array([a, b, c, n, m, alpha, beta])

# Update parameters using the iterative method
params = update_parameters(mu_exp_flat, v_exp_flat, p_exp_flat, params)

# Extract updated parameters
a, b, c, n, m, alpha, beta = params

print(f"Estimated Parameters:\na = {a}\nb = {b}\nc = {c}\nn = {n}\nm = {m}\nalpha = {alpha}\nbeta = {beta}")

# Function to calculate mu based on user input
def calculate_mu(v_T, p_N):
    mu = huemer_model(v_T, p_N, a, b, c, n, m, alpha, beta)
    return mu

# Calculate mu for all experimental data points using the updated parameters
mu_pred = huemer_model(v_exp_flat, p_exp_flat, a, b, c, n, m, alpha, beta)

# Calculate MAPE
mape = np.mean(np.abs((mu_pred - mu_exp_flat) / mu_exp_flat)) * 100

#input
v_T = float(input("Enter sliding velocity v_T (mm/s): "))
p_N = float(input("Enter normal pressure p_N (MPa): "))

mu = calculate_mu(v_T, p_N)
print(f"Calculated friction coefficient (mu) for v_T={v_T} mm/s and p_N={p_N} MPa is: {mu}")

# Calculate the friction coefficients for all combinations of VT and PN using the optimized parameters
print("\nFriction coefficients for all combinations of VT and PN using optimized parameters:")
for i, vt in enumerate(v_exp):
    for j, pn in enumerate(p_exp):
        mu_calc = calculate_mu(vt, pn)
        print(f"VT={vt} mm/s, PN={pn} MPa: {mu_calc:.4f}")

print(f"Overall Mean Absolute Percentage Error (MAPE): {mape:.4f}%")


Estimated Parameters:
a = 1.7809311116097442
b = 3.0609562205654774
c = 2.6434843535661177
n = 0.633158309993741
m = 5.055951180048124
alpha = 1.1791325893711784
beta = 0.9050209861463481
Enter sliding velocity v_T (mm/s): 1
Enter normal pressure p_N (MPa): 100
Calculated friction coefficient (mu) for v_T=1.0 mm/s and p_N=100.0 MPa is: 0.23630925189573085

Friction coefficients for all combinations of VT and PN using optimized parameters:
VT=1 mm/s, PN=0.1 MPa: 0.7681
VT=1 mm/s, PN=0.5 MPa: 0.5105
VT=1 mm/s, PN=1.0 MPa: 0.4387
VT=1 mm/s, PN=1.5 MPa: 0.4044
VT=1 mm/s, PN=2.0 MPa: 0.3829
VT=10 mm/s, PN=0.1 MPa: 1.0863
VT=10 mm/s, PN=0.5 MPa: 0.7220
VT=10 mm/s, PN=1.0 MPa: 0.6204
VT=10 mm/s, PN=1.5 MPa: 0.5719
VT=10 mm/s, PN=2.0 MPa: 0.5416
VT=100 mm/s, PN=0.1 MPa: 1.2080
VT=100 mm/s, PN=0.5 MPa: 0.8029
VT=100 mm/s, PN=1.0 MPa: 0.6899
VT=100 mm/s, PN=1.5 MPa: 0.6360
VT=100 mm/s, PN=2.0 MPa: 0.6023
VT=1000 mm/s, PN=0.1 MPa: 1.1285
VT=1000 mm/s, PN=0.5 MPa: 0.7501
VT=1000 mm/s, PN=1.0 MPa: 

OPTIMIZATION METHOD: BASIN HOPPING


In [ ]:
#BASIN HOPPING
import numpy as np
from scipy.optimize import minimize, basinhopping

# Define the function to fit
def huemer_friction_law(vt_pn, a, b, c, n, m, alpha, beta):
    vt, pn = vt_pn
    return (alpha * np.abs(pn)**(n-1) + beta) / (a + b/(np.abs(vt)**(1/m)) + c/(np.abs(vt)**(2/m)))

# Read data from external file
with open('/content/data.txt', 'r') as file:
    lines = file.readlines()

# Extract v_values, P_values, and mu_data
v_values = np.array([float(val) for val in lines[0].split(':')[1].split()])
P_values = np.array([float(val) for val in lines[1].split(':')[1].split()])
mu_data = np.array([[float(val) for val in line.split()] for line in lines[3:]])

# Create meshgrid of VT and PN
VT, PN = np.meshgrid(v_values, P_values)

# Prepare the dependent variable (mu)
mu = mu_data.T.ravel()

# Define objective function for optimization
def objective_function(params):
    a, b, c, n, m, alpha, beta = params
    mu_calc = huemer_friction_law((VT.flatten(), PN.flatten()), a, b, c, n, m, alpha, beta)
    absolute_percentage_error = np.abs((mu_calc - mu) / mu) * 100
    return np.mean(absolute_percentage_error)

# Define bounds for parameters
bounds = [(1.0, 10.0), (-10.0, 1.0), (1.0, 10.0), (1.0, 10.0), (1.0, 10.0), (-10.0, -1.0), (1.0, 10.0)]

# Perform optimization using Basin Hopping with bounds
result_bh = basinhopping(objective_function, x0=np.random.uniform(0, 10, size=7),
                         niter=100, minimizer_kwargs={"method": "L-BFGS-B", "bounds": bounds})

# Use optimized parameters from Basin Hopping
popt_bh = result_bh.x

# Print the optimal parameter values
print("Optimal parameter values after optimization (Basin Hopping method):")
print("a = {:.4f}, b = {:.4f}, c = {:.4f}, n = {:.4f}, m = {:.4f}, alpha = {:.4f}, beta = {:.4f}".format(*popt_bh))

# Input VT and PN values
vt_input = float(input("\nEnter the value for VT (mm/s): "))
pn_input = float(input("Enter the value for PN (MPa): "))

# Calculate the friction coefficient for the input VT and PN using optimized parameters
calculated_mu_input = huemer_friction_law((vt_input, pn_input), *popt_bh)
print("\nFriction coefficient at VT={:.1f} mm/s and PN={:.1f} MPa: {:.4f}".format(vt_input, pn_input, calculated_mu_input))

# Find the index of the closest VT and PN values in the data
vt_index = np.argmin(np.abs(v_values - vt_input))
pn_index = np.argmin(np.abs(P_values - pn_input))

# Get the corresponding mu value from the data
actual_mu = mu_data[vt_index, pn_index]

# Calculate MAPE between calculated and actual mu values
mape = np.abs((calculated_mu_input - actual_mu) / actual_mu) * 100

# Calculate the friction coefficients for all combinations of VT and PN using the optimized parameters from Basin Hopping
calculated_mu_bh = huemer_friction_law((VT, PN), *popt_bh)

# Print the mu values for all combinations of VT and PN
print("\nMu values for all combinations of VT and PN (Basin Hopping method):")
for i, vt in enumerate(v_values):
    for j, pn in enumerate(P_values):
        print(f"VT={vt} mm/s, PN={pn} MPa: {calculated_mu_bh[j, i]:.4f}")

# Calculate MAPE for all data points
# Calculate MAPE for all data points
mape_array = np.abs((calculated_mu_bh.T - mu_data) / mu_data) * 100

# Calculate the overall MAPE
overall_mape = np.mean(mape_array)

# Print the overall MAPE
print("\nOverall Mean Absolute Percentage Error (MAPE) for all data points: {:.4f}%".format(overall_mape))




Optimal parameter values after optimization (Basin Hopping method):
a = 4.9207, b = -4.4223, c = 5.2974, n = 1.1321, m = 8.4003, alpha = -6.0178, beta = 8.8227

Enter the value for VT (mm/s): 1000
Enter the value for PN (MPa): 2.0

Friction coefficient at VT=1000.0 mm/s and PN=2.0 MPa: 0.5569

Mu values for all combinations of VT and PN (Basin Hopping method):
VT=1.0 mm/s, PN=0.1 MPa: 0.7563
VT=1.0 mm/s, PN=0.5 MPa: 0.5748
VT=1.0 mm/s, PN=1.0 MPa: 0.4839
VT=1.0 mm/s, PN=1.5 MPa: 0.4268
VT=1.0 mm/s, PN=2.0 MPa: 0.3844
VT=10.0 mm/s, PN=0.1 MPa: 0.9487
VT=10.0 mm/s, PN=0.5 MPa: 0.7210
VT=10.0 mm/s, PN=1.0 MPa: 0.6071
VT=10.0 mm/s, PN=1.5 MPa: 0.5354
VT=10.0 mm/s, PN=2.0 MPa: 0.4822
VT=100.0 mm/s, PN=0.1 MPa: 1.0602
VT=100.0 mm/s, PN=0.5 MPa: 0.8058
VT=100.0 mm/s, PN=1.0 MPa: 0.6784
VT=100.0 mm/s, PN=1.5 MPa: 0.5983
VT=100.0 mm/s, PN=2.0 MPa: 0.5388
VT=1000.0 mm/s, PN=0.1 MPa: 1.0957
VT=1000.0 mm/s, PN=0.5 MPa: 0.8328
VT=1000.0 mm/s, PN=1.0 MPa: 0.7012
VT=1000.0 mm/s, PN=1.5 MPa: 0.6184
VT

OPTIMIZATION TECHNIQUE: DIFFERENTIAL EVOLUTION


In [ ]:
#DIFFERENTIAL EVOLUTION
import numpy as np
from scipy.optimize import minimize, differential_evolution

# Define the function to fit
def huemer_friction_law(vt_pn, a, b, c, n, m, alpha, beta):
    vt, pn = vt_pn
    return (alpha * np.abs(pn)**(n-1) + beta) / (a + b/(np.abs(vt)**(1/m)) + c/(np.abs(vt)**(2/m)))

# Read data from external file
with open('/content/data.txt', 'r') as file:
    lines = file.readlines()

# Extract v_values, P_values, and mu_data
v_values = np.array([float(val) for val in lines[0].split(':')[1].split()])
P_values = np.array([float(val) for val in lines[1].split(':')[1].split()])
mu_data = np.array([[float(val) for val in line.split()] for line in lines[3:]])

# Create meshgrid of VT and PN
VT, PN = np.meshgrid(v_values, P_values)

# Prepare the dependent variable (mu)
mu = mu_data.T.ravel()

# Define objective function for optimization
def objective_function(params):
    a, b, c, n, m, alpha, beta = params
    mu_calc = huemer_friction_law((VT.flatten(), PN.flatten()), a, b, c, n, m, alpha, beta)
    absolute_percentage_error = np.abs((mu_calc - mu) / mu) * 100
    return np.mean(absolute_percentage_error)

# Define bounds for parameters
bounds = [(1.0, 10.0), (-10.0, 1.0), (1.0, 10.0), (1.0, 10.0), (1.0, 10.0), (-10.0, -1.0), (1.0, 10.0)]
# Set random seed for reproducibility
np.random.seed(42)
# Perform optimization using Differential Evolution with bounds and strategy 'best1bin'
result_de = differential_evolution(objective_function, bounds, strategy='best1bin', popsize=30, tol=1e-4)
# Use optimized parameters from Differential Evolution
popt_de = result_de.x
# Perform optimization using L-BFGS-B method with bounds
result_lbfgsb = minimize(objective_function, x0=popt_de, method='L-BFGS-B', bounds=bounds, tol=1e-6)
# Use optimized parameters from L-BFGS-B method
popt_lbfgsb = result_lbfgsb.x
# Print the optimal parameters
print("Optimal parameters after optimization:")
print("a = {:.4f}, b = {:.4f}, c = {:.4f}, n = {:.4f}, m = {:.4f}, alpha = {:.4f}, beta = {:.4f}".format(*popt_lbfgsb))

# Input VT and PN values
vt_input = float(input("Enter the value for VT (mm/s): "))
pn_input = float(input("Enter the value for PN (MPa): "))

# Calculate the friction coefficient for the input VT and PN using optimized parameters from L-BFGS-B method
calculated_mu_input = huemer_friction_law((vt_input, pn_input), *popt_lbfgsb)
print("Friction coefficient at VT={:.1f} mm/s and PN={:.1f} MPa: {:.4f}".format(vt_input, pn_input, calculated_mu_input))

# Find the index of the closest VT and PN values in the data
vt_index = np.argmin(np.abs(v_values - vt_input))
pn_index = np.argmin(np.abs(P_values - pn_input))

# Get the corresponding mu value from the data
actual_mu = mu_data[vt_index, pn_index]

# Calculate MAPE between calculated and actual mu values
mape = np.abs((calculated_mu_input - actual_mu) / actual_mu) * 100

# Print all mu values for respective VT and PN
print("\nCalculated mu values for all combinations of VT and PN:")
for i, vt in enumerate(v_values):
    for j, pn in enumerate(P_values):
        mu_calc = huemer_friction_law((vt, pn), *popt_lbfgsb)
        print("VT={:.1f} mm/s, PN={:.1f} MPa: Calculated Mu = {:.4f}".format(vt, pn, mu_calc))

# Calculate MAPE between calculated and actual mu values for all data points
mape_array = np.abs((huemer_friction_law((VT.flatten(), PN.flatten()), *popt_lbfgsb) - mu_data.T.ravel()) / mu_data.T.ravel()) * 100
overall_mape = np.mean(mape_array)

# Print the overall MAPE
print("\nOverall Mean Absolute Percentage Error (MAPE) for all data points: {:.4f}%".format(overall_mape))





Optimal parameters after optimization:
a = 1.0042, b = -1.1221, c = 1.2305, n = 1.0312, m = 9.9956, alpha = -5.1192, beta = 5.6492
Enter the value for VT (mm/s): 100
Enter the value for PN (MPa): 0.5
Friction coefficient at VT=100.0 mm/s and PN=0.5 MPa: 0.8135

Calculated mu values for all combinations of VT and PN:
VT=1.0 mm/s, PN=0.1 MPa: Calculated Mu = 0.7951
VT=1.0 mm/s, PN=0.5 MPa: Calculated Mu = 0.5747
VT=1.0 mm/s, PN=1.0 MPa: Calculated Mu = 0.4764
VT=1.0 mm/s, PN=1.5 MPa: Calculated Mu = 0.4178
VT=1.0 mm/s, PN=2.0 MPa: Calculated Mu = 0.3759
VT=10.0 mm/s, PN=0.1 MPa: Calculated Mu = 0.9949
VT=10.0 mm/s, PN=0.5 MPa: Calculated Mu = 0.7191
VT=10.0 mm/s, PN=1.0 MPa: Calculated Mu = 0.5960
VT=10.0 mm/s, PN=1.5 MPa: Calculated Mu = 0.5228
VT=10.0 mm/s, PN=2.0 MPa: Calculated Mu = 0.4703
VT=100.0 mm/s, PN=0.1 MPa: Calculated Mu = 1.1255
VT=100.0 mm/s, PN=0.5 MPa: Calculated Mu = 0.8135
VT=100.0 mm/s, PN=1.0 MPa: Calculated Mu = 0.6743
VT=100.0 mm/s, PN=1.5 MPa: Calculated Mu = 0.59

OPTIMIZATION TECHNIQUE: CONJUGATE GRADIENT

In [ ]:
#CONJUGATE GRADIENT METHOD
import numpy as np
from scipy.optimize import minimize

# Define the function to fit
def huemer_friction_law(vt_pn, a, b, c, n, m, alpha, beta):
    vt, pn = vt_pn
    return (alpha * np.abs(pn)**(n-1) + beta) / (a + b/(np.abs(vt)**(1/m)) + c/(np.abs(vt)**(2/m)))

# Read data from external file
with open('/content/data.txt', 'r') as file:
    lines = file.readlines()

# Extract v_values, P_values, and mu_data
v_values = np.array([float(val) for val in lines[0].split(':')[1].split()])
P_values = np.array([float(val) for val in lines[1].split(':')[1].split()])
mu_data = np.array([[float(val) for val in line.split()] for line in lines[3:]])

# Create meshgrid of VT and PN
VT, PN = np.meshgrid(v_values, P_values)

# Prepare the dependent variable (mu)
mu = mu_data.T.ravel()

# Define objective function for optimization
def objective_function(params):
    a, b, c, n, m, alpha, beta = params
    mu_calc = huemer_friction_law((VT.flatten(), PN.flatten()), a, b, c, n, m, alpha, beta)
    absolute_percentage_error = np.abs((mu_calc - mu) / mu) * 100
    return np.mean(absolute_percentage_error)
# Define initial guess for parameters
initial_guess = [1.0, -1.0, 1.0, 1.0, 1.0, -1.0, 1.0]
# Perform optimization using Conjugate Gradient (CG) method
result_cg = minimize(objective_function, initial_guess, method='CG')
# Use optimized parameters from CG method
popt_cg = result_cg.x

# Print the optimal parameter values
print("Optimal parameter values after optimization (Conjugate Gradient method):")
print("a = {:.4f}, b = {:.4f}, c = {:.4f}, n = {:.4f}, m = {:.4f}, alpha = {:.4f}, beta = {:.4f}".format(*popt_cg))

# Input VT and PN values
vt_input = float(input("\nEnter the value for VT (mm/s): "))
pn_input = float(input("Enter the value for PN (MPa): "))

# Calculate the friction coefficient for the input VT and PN using optimized parameters
calculated_mu_input = huemer_friction_law((vt_input, pn_input), *popt_cg)
print("\nFriction coefficient at VT={:.1f} mm/s and PN={:.1f} MPa: {:.4f}".format(vt_input, pn_input, calculated_mu_input))

# Find the index of the closest VT and PN values in the data
vt_index = np.argmin(np.abs(v_values - vt_input))
pn_index = np.argmin(np.abs(P_values - pn_input))

# Get the corresponding mu value from the data
actual_mu = mu_data[vt_index, pn_index]

# Calculate MAPE between calculated and actual mu values
mape = np.abs((calculated_mu_input - actual_mu) / actual_mu) * 100

# Calculate the friction coefficients for all combinations of VT and PN using the optimized parameters from CG method
calculated_mu_cg = huemer_friction_law((VT, PN), *popt_cg)

# Print the mu values for all combinations of VT and PN
print("\nMu values for all combinations of VT and PN (Conjugate Gradient method):")
for i, vt in enumerate(v_values):
    for j, pn in enumerate(P_values):
        print(f"VT={vt} mm/s, PN={pn} MPa: {calculated_mu_cg[j, i]:.4f}")

# Calculate MAPE for all data points
mape_array = np.abs((huemer_friction_law((VT.flatten(), PN.flatten()), *popt_cg) - mu_data.T.ravel()) / mu_data.T.ravel()) * 100

# Calculate the overall MAPE
overall_mape = np.mean(mape_array)

# Print the overall MAPE
print("\nOverall Mean Absolute Percentage Error (MAPE) for all data points: {:.4f}%".format(overall_mape))


Optimal parameter values after optimization (Conjugate Gradient method):
a = 0.9898, b = -0.7806, c = 1.1838, n = 1.3327, m = 0.9486, alpha = -0.6631, beta = 1.3262

Enter the value for VT (mm/s): 1000
Enter the value for PN (MPa): 2.0

Friction coefficient at VT=1000.0 mm/s and PN=2.0 MPa: 0.4965

Mu values for all combinations of VT and PN (Conjugate Gradient method):
VT=1.0 mm/s, PN=0.1 MPa: 0.7308
VT=1.0 mm/s, PN=0.5 MPa: 0.5741
VT=1.0 mm/s, PN=1.0 MPa: 0.4760
VT=1.0 mm/s, PN=1.5 MPa: 0.4073
VT=1.0 mm/s, PN=2.0 MPa: 0.3526
VT=10.0 mm/s, PN=0.1 MPa: 1.0945
VT=10.0 mm/s, PN=0.5 MPa: 0.8598
VT=10.0 mm/s, PN=1.0 MPa: 0.7129
VT=10.0 mm/s, PN=1.5 MPa: 0.6100
VT=10.0 mm/s, PN=2.0 MPa: 0.5280
VT=100.0 mm/s, PN=0.1 MPa: 1.0348
VT=100.0 mm/s, PN=0.5 MPa: 0.8129
VT=100.0 mm/s, PN=1.0 MPa: 0.6740
VT=100.0 mm/s, PN=1.5 MPa: 0.5767
VT=100.0 mm/s, PN=2.0 MPa: 0.4992
VT=1000.0 mm/s, PN=0.1 MPa: 1.0290
VT=1000.0 mm/s, PN=0.5 MPa: 0.8084
VT=1000.0 mm/s, PN=1.0 MPa: 0.6703
VT=1000.0 mm/s, PN=1.5 MPa:

OPTIMIZATION TECHNIQUE: POWELL METHOD(DE)

In [ ]:
#POWEL TECHNIQUE
import numpy as np
from scipy.optimize import minimize, differential_evolution

# Define the function to fit
def huemer_friction_law(vt_pn, a, b, c, n, m, alpha, beta):
    vt, pn = vt_pn
    return (alpha * np.abs(pn)**(n-1) + beta) / (a + b/(np.abs(vt)**(1/m)) + c/(np.abs(vt)**(2/m)))

# Read data from external file
with open('/content/data.txt', 'r') as file:
    lines = file.readlines()

# Extract v_values, P_values, and mu_data
v_values = np.array([float(val) for val in lines[0].split(':')[1].split()])
P_values = np.array([float(val) for val in lines[1].split(':')[1].split()])
mu_data = np.array([[float(val) for val in line.split()] for line in lines[3:]])

# Create meshgrid of VT and PN
VT, PN = np.meshgrid(v_values, P_values)

# Prepare the dependent variable (mu)
mu = mu_data.T.ravel()

# Define objective function for optimization
def objective_function(params):
    a, b, c, n, m, alpha, beta = params
    mu_calc = huemer_friction_law((VT.flatten(), PN.flatten()), a, b, c, n, m, alpha, beta)
    absolute_percentage_error = np.abs((mu_calc - mu) / mu) * 100
    return np.mean(absolute_percentage_error)

# Define bounds for parameters
bounds = [(1.0, 10.0), (-10.0, 1.0), (1.0, 10.0), (1.0, 10.0), (1.0, 10.0), (-10.0, -1.0), (1.0, 10.0)]

# Perform optimization using Differential Evolution with bounds and strategy 'best1bin'
result_de = differential_evolution(objective_function, bounds, strategy='best1bin')

# Use optimized parameters from Differential Evolution
popt_de = result_de.x

# Print the optimal parameter values
print("Optimal parameter values after optimization:")
print("a = {:.4f}, b = {:.4f}, c = {:.4f}, n = {:.4f}, m = {:.4f}, alpha = {:.4f}, beta = {:.4f}".format(*popt_de))

# Perform optimization using Powell method with bounds
result_powell = minimize(objective_function, x0=popt_de, method='Powell', bounds=bounds)

# Use optimized parameters from Powell method
popt_powell = result_powell.x

# Input VT and PN values
vt_input = float(input("Enter the value for VT (mm/s): "))
pn_input = float(input("Enter the value for PN (MPa): "))

# Calculate the friction coefficient for the input VT and PN using optimized parameters
calculated_mu_input = huemer_friction_law((vt_input, pn_input), *popt_powell)
print("Friction coefficient at VT={:.1f} mm/s and PN={:.1f} MPa: {:.4f}".format(vt_input, pn_input, calculated_mu_input))

# Find the index of the closest VT and PN values in the data
vt_index = np.argmin(np.abs(v_values - vt_input))
pn_index = np.argmin(np.abs(P_values - pn_input))

# Get the corresponding mu value from the data
actual_mu = mu_data[vt_index, pn_index]

# Calculate MAPE between calculated and actual mu values
mape = np.abs((calculated_mu_input - actual_mu) / actual_mu) * 100


# Calculate the friction coefficients for all combinations of VT and PN using the optimized parameters from Powell method
calculated_mu_powell = huemer_friction_law((VT, PN), *popt_powell)

# Print the mu values for all combinations of VT and PN
print("Mu values for all combinations of VT and PN:")
for i, vt in enumerate(v_values):
    for j, pn in enumerate(P_values):
        print(f"VT={vt} mm/s, PN={pn} MPa: {calculated_mu_powell[j, i]:.4f}")

# Calculate MAPE for all data points
mu_calc_powell = huemer_friction_law((VT.flatten(), PN.flatten()), *popt_powell)
mape_array_powell = np.abs((mu_calc_powell - mu_data.T.ravel()) / mu_data.T.ravel()) * 100

# Calculate the overall MAPE
overall_mape_powell = np.mean(mape_array_powell)

# Print the overall MAPE
print("\nOverall Mean Absolute Percentage Error (MAPE) for all data points (Powell method): {:.4f}%".format(overall_mape_powell))



Optimal parameter values after optimization:
a = 5.1006, b = -4.4807, c = 5.6411, n = 1.1372, m = 8.7552, alpha = -6.4722, beta = 9.4733
Enter the value for VT (mm/s): 1000
Enter the value for PN (MPa): 2.0
Friction coefficient at VT=1000.0 mm/s and PN=2.0 MPa: 0.5569
Mu values for all combinations of VT and PN:
VT=1.0 mm/s, PN=0.1 MPa: 0.7594
VT=1.0 mm/s, PN=0.5 MPa: 0.5731
VT=1.0 mm/s, PN=1.0 MPa: 0.4793
VT=1.0 mm/s, PN=1.5 MPa: 0.4202
VT=1.0 mm/s, PN=2.0 MPa: 0.3762
VT=10.0 mm/s, PN=0.1 MPa: 0.9528
VT=10.0 mm/s, PN=0.5 MPa: 0.7191
VT=10.0 mm/s, PN=1.0 MPa: 0.6014
VT=10.0 mm/s, PN=1.5 MPa: 0.5272
VT=10.0 mm/s, PN=2.0 MPa: 0.4720
VT=100.0 mm/s, PN=0.1 MPa: 1.0750
VT=100.0 mm/s, PN=0.5 MPa: 0.8113
VT=100.0 mm/s, PN=1.0 MPa: 0.6785
VT=100.0 mm/s, PN=1.5 MPa: 0.5948
VT=100.0 mm/s, PN=2.0 MPa: 0.5325
VT=1000.0 mm/s, PN=0.1 MPa: 1.1242
VT=1000.0 mm/s, PN=0.5 MPa: 0.8485
VT=1000.0 mm/s, PN=1.0 MPa: 0.7096
VT=1000.0 mm/s, PN=1.5 MPa: 0.6220
VT=1000.0 mm/s, PN=2.0 MPa: 0.5569

Overall Mean Ab

OPTIMIZATION TECHNIQUE: NELDER-MEAD



In [ ]:
#NELDER-MEAD
import numpy as np
from scipy.optimize import minimize

# Define the function to fit
def huemer_friction_law(vt_pn, a, b, c, n, m, alpha, beta):
    vt, pn = vt_pn
    return (alpha * np.abs(pn)**(n-1) + beta) / (a + b/(np.abs(vt)**(1/m)) + c/(np.abs(vt)**(2/m)))

# Read data from external file
with open('/content/data.txt', 'r') as file:
    lines = file.readlines()

# Extract v_values, P_values, and mu_data
v_values = np.array([float(val) for val in lines[0].split(':')[1].split()])
P_values = np.array([float(val) for val in lines[1].split(':')[1].split()])
mu_data = np.array([[float(val) for val in line.split()] for line in lines[3:]])

# Create meshgrid of VT and PN
VT, PN = np.meshgrid(v_values, P_values)

# Prepare the dependent variable (mu)
mu = mu_data.T.ravel()

# Define the objective function for optimization
def objective_function(params):
    a, b, c, n, m, alpha, beta = params
    mu_calc = huemer_friction_law((VT.flatten(), PN.flatten()), a, b, c, n, m, alpha, beta)
    absolute_percentage_error = np.abs((mu_calc - mu) / mu) * 100
    return np.mean(absolute_percentage_error)

# Define initial guess for parameters
initial_guess = [5.0, -5.0, 5.0, 5.0, 5.0, -5.0, 5.0]

# Perform optimization using Nelder-Mead
result = minimize(objective_function, initial_guess, method='Nelder-Mead')

# Extract optimal parameter values
popt = result.x

# Print the optimal parameter values
print("Optimal parameter values after Nelder-Mead optimization:")
print("a = {:.4f}, b = {:.4f}, c = {:.4f}, n = {:.4f}, m = {:.4f}, alpha = {:.4f}, beta = {:.4f}".format(*popt))

# Print the MAPE with optimized parameters
final_mape = result.fun

# Input VT and PN values
vt_input = float(input("Enter the value for VT (mm/s): "))
pn_input = float(input("Enter the value for PN (MPa): "))

# Calculate the friction coefficient for the input VT and PN using optimized parameters
calculated_mu_input = huemer_friction_law((vt_input, pn_input), *popt)
print("Friction coefficient at VT={:.1f} mm/s and PN={:.1f} MPa: {:.4f}".format(vt_input, pn_input, calculated_mu_input))

# Find the index of the closest VT and PN values in the data
vt_index = np.argmin(np.abs(v_values - vt_input))
pn_index = np.argmin(np.abs(P_values - pn_input))

# Get the corresponding mu value from the data
actual_mu = mu_data[vt_index, pn_index]

# Calculate MAPE between calculated and actual mu values
mape = np.abs((calculated_mu_input - actual_mu) / actual_mu) * 100
print("Mean Absolute Percentage Error (MAPE) between calculated and actual mu values: {:.4f}%".format(mape))

# Calculate the friction coefficients for all combinations of VT and PN using the optimized parameters from Nelder-Mead
calculated_mu_nelder = huemer_friction_law((VT, PN), *popt)

# Print the mu values for all combinations of VT and PN
print("\nFriction coefficients for all combinations of VT and PN using Nelder-Mead optimized parameters:")
for i, vt in enumerate(v_values):
    for j, pn in enumerate(P_values):
        calculated_mu = calculated_mu_nelder[j, i]
        print("VT={:.1f} mm/s, PN={:.1f} MPa: {:.4f}".format(vt, pn, calculated_mu))

# Calculate MAPE for all data points
mu_calc_nelder = huemer_friction_law((VT.flatten(), PN.flatten()), *popt)
mape_array_nelder = np.abs((mu_calc_nelder - mu_data.T.ravel()) / mu_data.T.ravel()) * 100

# Calculate the overall MAPE
overall_mape_nelder = np.mean(mape_array_nelder)

# Print the overall MAPE
print("\nOverall Mean Absolute Percentage Error (MAPE) for all data points (Nelder-Mead method): {:.4f}%".format(overall_mape_nelder))


Optimal parameter values after Nelder-Mead optimization:
a = 5.7116, b = -5.1040, c = 6.2324, n = 1.2766, m = 6.2539, alpha = -3.5906, beta = 6.8895
Enter the value for VT (mm/s): 1000
Enter the value for PN (MPa): 2.0
Friction coefficient at VT=1000.0 mm/s and PN=2.0 MPa: 0.5399
Mean Absolute Percentage Error (MAPE) between calculated and actual mu values: 3.0561%

Friction coefficients for all combinations of VT and PN using Nelder-Mead optimized parameters:
VT=1.0 mm/s, PN=0.1 MPa: 0.7296
VT=1.0 mm/s, PN=0.5 MPa: 0.5739
VT=1.0 mm/s, PN=1.0 MPa: 0.4823
VT=1.0 mm/s, PN=1.5 MPa: 0.4200
VT=1.0 mm/s, PN=2.0 MPa: 0.3713
VT=10.0 mm/s, PN=0.1 MPa: 0.9664
VT=10.0 mm/s, PN=0.5 MPa: 0.7601
VT=10.0 mm/s, PN=1.0 MPa: 0.6388
VT=10.0 mm/s, PN=1.5 MPa: 0.5563
VT=10.0 mm/s, PN=2.0 MPa: 0.4918
VT=100.0 mm/s, PN=0.1 MPa: 1.0626
VT=100.0 mm/s, PN=0.5 MPa: 0.8358
VT=100.0 mm/s, PN=1.0 MPa: 0.7024
VT=100.0 mm/s, PN=1.5 MPa: 0.6117
VT=100.0 mm/s, PN=2.0 MPa: 0.5408
VT=1000.0 mm/s, PN=0.1 MPa: 1.0608
VT=10